In [20]:
import numpy as np
import time
from grid_world import standard_grid,negative_grid
SMALL_TOL=10e-8
ALL_Actions=['U','D','L','R',]
def print_values(V, g):
  print("Values:")
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")


def print_policy(P, g):
  print("Policy:")
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      a = P.get((i,j), ' ')
      print("  %s  |" % a, end="")
    print("")
    

In [7]:
# Main starts here

In [8]:
grid = negative_grid()
print_values(grid.rewards,grid) #check the reward matrix

---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [42]:
## initialize random policies
policy={}
for s in grid.actions.keys():
    policy[s]=np.random.choice(ALL_Actions) # illegal actions will not move
print('initial policy')
print_policy(policy,grid)

### uniformly random actions ###
# initialize V(s) = 0
V = {}
states=grid.all_states()
for s in states:
    V[s] = 0
    if s in grid.actions:
        V[s]=np.random.random()
gamma = 0.9 # discount factor
# print_values(V,grid)

# combine two iterations together
iter_n=0
while True:
    #policy eval
    while True:
        biggest_change = 0
        for s in states:
            old_v = V[s]
            # V(s) only has value if it's not a terminal state
            if s in policy:
                a = policy[s]
                grid.set_state(s)
                r = grid.move(a)
                V[s] = r + gamma * V[grid.current_state()]
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        if biggest_change < SMALL_TOL: # check is stopping criteria met
            break
    #policy improvement
    is_policy_converged=True
    for s in states:
        if s in policy:
            old_a=policy[s]
            new_a=None
            best_value=float('-inf') # assign the smallest possible value in python
            for a in ALL_Actions:
                grid.set_state(s)
                r=grid.move(a)
                v=r+gamma*V[grid.current_state()]
                if v>best_value:
                    best_value=v
                    new_a=a
            policy[s]=new_a
            if new_a!=old_a:
                is_policy_converged=False
    #check if end                
    if is_policy_converged:
        break
    iter_n+=1
    print(iter_n,end="\r")
    time.sleep(0.1)
    
#check the result
print()
print_values(V,grid)
print_policy(policy,grid)

initial policy
Policy:
---------------------------
  L  |  U  |  L  |     |
---------------------------
  L  |     |  L  |     |
---------------------------
  D  |  R  |  R  |  R  |
4
Values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|
Policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
